# 测试《天龙八部》rerank效果

## 准备

In [1]:
%%time

from llama_index.core import SummaryIndex, Document
from llama_index.core import Settings

from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

from milvus import default_server
from pymilvus import connections, utility

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.core.llms import ChatMessage

CPU times: user 3.58 s, sys: 460 ms, total: 4.04 s
Wall time: 3.67 s


In [2]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f04d7979150>, id_func=<function default_id_func at 0x7f059d57af80>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [3]:
%%time

# 启动milvus服务器

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

# Stop your milvus server
# default_server.stop()

default_server.listen_port

v2.3.5-lite
CPU times: user 3.17 s, sys: 300 ms, total: 3.47 s
Wall time: 6.58 s


19530

In [4]:
%%time

# 初始化全局 llm

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 117 ms, sys: 3.68 ms, total: 121 ms
Wall time: 122 ms


In [5]:
%%time

# 初始化全局 embedding 模型

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 388 ms, sys: 23.8 ms, total: 412 ms
Wall time: 411 ms


In [6]:
%%time

import logging
import sys

# 设置系统日志，便于设置level排查

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 353 µs, sys: 0 ns, total: 353 µs
Wall time: 356 µs


In [7]:
%%time

# load documents
documents = SimpleDirectoryReader(input_files=["books2/天龙八部.txt"]).load_data()
for doc in documents:
    print(doc.doc_id, doc.metadata['file_name'])

dc73b62a-ebeb-4786-9a64-a167b6c27f79 天龙八部.txt
CPU times: user 4.31 ms, sys: 8.05 ms, total: 12.4 ms
Wall time: 11.8 ms


In [8]:
%%time

vector_store = MilvusVectorStore(dim=1024, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: f229448f62214eb0ae1a463aeace41d8
Created new connection using: f229448f62214eb0ae1a463aeace41d8
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection
Successfully created an index on collection: llamacollection
CPU times: user 1min 24s, sys: 2.9 s, total: 1min 27s
Wall time: 7min 32s


## 基于基本嵌入问答

In [26]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("虚竹的妈妈是谁，说出她的名字")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
在《天龙八部》的故事里，虚竹的母亲名叫叶二娘。
CPU times: user 132 ms, sys: 16.8 ms, total: 149 ms
Wall time: 11.1 s


## 基于rerank的问答

In [10]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 934 ms, sys: 669 ms, total: 1.6 s
Wall time: 806 ms


In [25]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

streaming_response = query_engine.query("虚竹的妈妈是谁，说出她的名字")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
叶二娘是虚竹的妈妈。
CPU times: user 696 ms, sys: 462 µs, total: 696 ms
Wall time: 1.37 s
